In [1]:
!nvidia-smi

Mon Jan 17 10:56:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro M4000        On   | 00000000:00:05.0 Off |                  N/A |
| 46%   26C    P8    11W / 120W |      1MiB /  8126MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install matplotlib pandas
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from zipfile import ZipFile
from CustomLayers import CustomConv2D, CustomConvTranspose2D

     |████████████████████████████████| 11.3 MB 21.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 890 kB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 27.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 31.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [74]:
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# physical_devices

In [4]:
!pip install gdown
import gdown
try:
    os.makedirs("celeba_gan")
except:
    pass

url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
path = "celeba_gan/data.zip"
gdown.download(url, path, quiet = True)

with ZipFile("celeba_gan/data.zip", "r") as zipobj:
    zipobj.extractall("celeba_gan")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 97 kB 10.6 MB/s eta 0:00:01
  Created wheel for gdown: filename=gdown-4.2.0-py3-none-any.whl size=14262 sha256=5d9e680a780d6934dcea52bc7d9244be2c360e3cad578561991b9780f6e17bd1
  Stored in directory: /root/.cache/pip/wheels/2b/3c/51/52c46deda5cd1d59c6ce3d441ea5f3d155495dc294c4535a25
Successfully built gdown
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [5]:
data_directory = "celeba_gan/img_align_celeba"
dataset = keras.preprocessing.image_dataset_from_directory(directory = data_directory,
                                                           label_mode = None,
                                                           image_size = (64, 64),
                                                           batch_size = 32,
                                                           shuffle = True).map(lambda x : x/255.0)

Found 202599 files belonging to 1 classes.


2022-01-17 11:01:22.174174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 11:01:22.187526: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 11:01:22.188499: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 11:01:22.190739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-17 11:01:22.191784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

### **Generator**

In [6]:
noise_dim = 128
input = layers.Input(shape = (noise_dim,))
dense = layers.Dense(8 * 8 * 1024)(input)
conv = layers.Reshape((8, 8, 1024))(dense)
conv_t = CustomConvTranspose2D(512, kernel_size = 5, strides = 2, padding = "same")(conv)
conv_t = layers.Conv2DTranspose(256, kernel_size = 5, strides = 2, padding = "same")(conv_t)
# conv_t = layers.Conv2DTranspose(128, kernel_size = 5, strides = 2, padding = "same")(conv_t)
output = layers.Conv2DTranspose(3, kernel_size = 5, strides = 2, padding = "same", activation = "sigmoid")(conv_t)

generator = keras.Model(inputs = input, outputs = output)
print(generator.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
dense (Dense)                (None, 65536)             8454144   
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 1024)        0         
_________________________________________________________________
custom_conv_transpose2d (Cus (None, 16, 16, 512)       13109760  
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       3277056   
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 3)         19203     
Total params: 24,860,163
Trainable params: 24,859,139
Non-trainable params: 1,024
_____________________________________________

### **Discriminator**

In [7]:
input = keras.Input(shape = (64, 64, 3))
conv = CustomConv2D(64, kernel_size = 4, strides = 2, padding = "same")(input)
conv = CustomConv2D(128, kernel_size = 4, strides = 2, padding = "same")(conv)
# conv = CustomConv2D(256, kernel_size = 4, strides = 2, padding = "same")(conv)
# conv = CustomConv2D(512, kernel_size = 4, strides = 2, padding = "same")(conv)
output = layers.Flatten()(conv)
output = layers.Dropout(0.2)(output)
output = layers.Dense(1, activation = "sigmoid")(output)

discriminator = keras.Model(inputs = input, outputs = output)
print(discriminator.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
custom_conv2d (CustomConv2D) (None, 32, 32, 64)        3392      
_________________________________________________________________
custom_conv2d_1 (CustomConv2 (None, 16, 16, 128)       131712    
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 32769     
Total params: 167,873
Trainable params: 167,489
Non-trainable params: 384
___________________________________________________

### **Optimisers**

In [8]:
opt_gen = keras.optimizers.Adam(2e-4)
opt_disc = keras.optimizers.Adam(2e-4)
loss_fn = keras.losses.BinaryCrossentropy()

### **Train Discriminator and Generator**

In [ ]:
from IPython.display import clear_output
import time
result_folder = str(time.time())
try:
    os.mkdir("models")
except:
    pass
try:
    generator = tf.keras.models.load_model(os.path.join("models", "generator"))
    discriminator = tf.keras.models.load_model(os.path.join("models", "discriminator"))
    print("Loading Models from Disk!")
except:
    print("Starting Training from Scratch!")

for epoch in range(10):
    for idx, real in enumerate(tqdm(dataset)):
        batch_size = real.shape[0]
        random_noise_vectors = tf.random.normal(shape = (batch_size, noise_dim))

        with tf.GradientTape(persistent = True) as tape:
            fake = generator(random_noise_vectors)
            output_disc_fake = discriminator(fake)
            output_disc_real = discriminator(real)
            
            loss_disc_real = loss_fn(tf.ones(batch_size, 1), output_disc_real)
            loss_disc_fake = loss_fn(tf.zeros(batch_size, 1), output_disc_fake)
            loss_disc = loss_disc_real + loss_disc_fake            
            loss_gen = loss_fn(tf.ones(batch_size, 1), output_disc_fake)
        
        if idx % 100 == 0:
            img = fake[0].numpy().astype('float32')
            result_name = f"{epoch}_{idx}_.png"
            result_path = "results"
            try:
                os.mkdir(result_path)
            except:
                pass
            try:
                os.mkdir(os.path.join(result_path, result_folder))
            except:
                pass
            result_path = os.path.join(result_path, result_folder)
            result_path = os.path.join(result_path, result_name)
            plt.imsave(result_path, img)
            generator.save(os.path.join("models", "generator"))
            discriminator.save(os.path.join("models", "discriminator"))
            clear_output()

        grads = tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(zip(grads, discriminator.trainable_weights))

        grads = tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(zip(grads, generator.trainable_weights))

Starting Training from Scratch!


  0%|          | 0/6332 [00:00<?, ?it/s]2022-01-17 11:01:34.889535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
